# 📋 초기 설정 및 해독표 정리

> **마스터 매핑 테이블을 분석하여 설문 요약본과 패널 정보 해독표를 생성하는 프로세스**

---

## 🎯 프로젝트 개요

| 항목 | 내용 |
|------|------|
| **목적** | 마스터 매핑 테이블을 분석하여 구조화된 해독표 생성 |
| **입력** | `../data/processed/00_master_mapping_table.csv` |
| **출력** | 설문 요약본 + 패널 정보 해독표 |
| **결과** | 분석에 필요한 구조화된 메타데이터 제공 |

## 📋 처리 과정

### 🔹 **설문 요약본 (Quickpoll Summary)**
- **대상**: Quickpoll 파일에서 추출된 설문 정보
- **내용**: 설문 제목, 참여자 수, 설문 기간 등
- **출력**: `quickpoll_summary.csv`

### 🔹 **패널 정보 해독표 (Panel Data Dictionary)**
- **대상**: Welcome 파일에서 추출된 패널 정보
- **내용**: 변수명, 설명, 타입, 코드-라벨 매핑
- **출력**: `panel_data_dictionary.csv`

---

## 📁 노트북 구조

1. **라이브러리 설정** - 필요한 패키지 import
2. **마스터 매핑 테이블 로드** - 원본 데이터 불러오기
3. **설문 요약본 생성** - Quickpoll 데이터 정리
4. **패널 정보 해독표 생성** - Welcome 데이터 정리
5. **결과 확인** - 생성된 파일들 검증

## 📚 1. 라이브러리 설정

데이터 처리에 필요한 라이브러리들을 import합니다.

In [40]:
import pandas as pd
import numpy as np
import os
import glob

## 📁 2. 마스터 매핑 테이블 로드

1단계에서 생성된 마스터 매핑 테이블을 불러옵니다.

In [41]:
# 1. 원본 마스터 매핑 테이블 불러오기
# 파일 경로를 사용자 환경에 맞게 확인해주세요.
try:
    master_df = pd.read_csv('../data/processed/00_master_mapping_table.csv', encoding='utf-8')
    print("✅ UTF-8 인코딩으로 파일을 성공적으로 불러왔습니다.")
except UnicodeDecodeError:
    master_df = pd.read_csv('../data/processed/00_master_mapping_table.csv', encoding='cp949')
    print("✅ CP949 인코딩으로 파일을 성공적으로 불러왔습니다.")

print(f"\n📊 마스터 매핑 테이블 정보:")
print(f"- 데이터 크기: {master_df.shape}")
print(f"- 컬럼 수: {len(master_df.columns)}")
print(f"- 출처 파일 수: {master_df['source_file'].nunique()}")

print(f"\n📋 컬럼 목록:")
for i, col in enumerate(master_df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n📁 출처 파일별 데이터 수:")
print(master_df['source_file'].value_counts())

✅ UTF-8 인코딩으로 파일을 성공적으로 불러왔습니다.

📊 마스터 매핑 테이블 정보:
- 데이터 크기: (652, 30)
- 컬럼 수: 30
- 출처 파일 수: 37

📋 컬럼 목록:
 1. 설문제목
 2. 보기1
 3. 보기2
 4. 보기3
 5. 보기4
 6. 보기5
 7. 보기6
 8. 보기7
 9. 보기8
10. 보기9
11. 보기10
12. 총참여자수
13. 보기1_CNT
14. 보기2_CNT
15. 보기3_CNT
16. 보기4_CNT
17. 보기5_CNT
18. 보기6_CNT
19. 보기7_CNT
20. 보기8_CNT
21. 보기9_CNT
22. 보기10_CNT
23. source_file
24. Unnamed: 12
25. Unnamed: 13
26. Unnamed: 14
27. Unnamed: 15
28. 변수명
29. 문항
30. 문항유형

📁 출처 파일별 데이터 수:
source_file
Welcome_2nd.xlsx          598
Welcome_1st.xlsx            7
qpoll_join_250317.xlsx      7
qpoll_join_250310.xlsx      5
qpoll_join_250304.xlsx      3
qpoll_join_250611.xlsx      1
qpoll_join_250703.xlsx      1
qpoll_join_250328.xlsx      1
qpoll_join_250627.xlsx      1
qpoll_join_250617.xlsx      1
qpoll_join_250723.xlsx      1
qpoll_join_250626.xlsx      1
qpoll_join_250221.xlsx      1
qpoll_join_250714.xlsx      1
qpoll_join_250702.xlsx      1
qpoll_join_250610.xlsx      1
qpoll_join_250709.xlsx      1
qpoll_join_250123.xlsx      1
q

## 📊 3. 설문 요약본 생성

Quickpoll 파일에서 추출된 설문 정보를 정리하여 설문 요약본을 생성합니다.

In [42]:
# 2. '설문 요약본 (Quickpoll Summary)' 추출 및 정리
print("=== 설문 요약본 생성 시작 ===")
quickpoll_df = master_df[master_df['source_file'].str.contains('qpoll', na=False)].copy()
print(f"Quickpoll 관련 데이터 수: {len(quickpoll_df)}개")

# 불필요한 컬럼 및 중간에 반복되는 헤더 행 제거
quickpoll_df = quickpoll_df.drop(columns=['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', '변수명', '문항', '문항유형'])
quickpoll_df = quickpoll_df.dropna(subset=['설문제목'])
quickpoll_df = quickpoll_df[quickpoll_df['설문제목'] != '설문제목']

# 데이터 타입 정리 (숫자여야 하는 컬럼들을 숫자로 변경)
count_cols = [col for col in quickpoll_df.columns if '_CNT' in col or '총참여자수' in col]
for col in count_cols:
    quickpoll_df[col] = pd.to_numeric(quickpoll_df[col], errors='coerce')

# 결과 저장
quickpoll_output_path = '../data/processed/quickpoll_summary.csv'
quickpoll_df.to_csv(quickpoll_output_path, index=False, encoding='utf-8-sig')

print(f"✅ 설문 요약본 저장 완료: {quickpoll_output_path}")
print(f"📊 최종 데이터 크기: {quickpoll_df.shape}")

# 데이터 미리보기
print(f"📋 설문 요약본 미리보기:")
display(quickpoll_df.head())

=== 설문 요약본 생성 시작 ===
Quickpoll 관련 데이터 수: 47개
✅ 설문 요약본 저장 완료: ../data/processed/quickpoll_summary.csv
📊 최종 데이터 크기: (41, 23)
📋 설문 요약본 미리보기:


,설문제목,보기1,보기2,보기3,보기4,보기5,보기6,보기7,보기8,보기9,...,보기2_CNT,보기3_CNT,보기4_CNT,보기5_CNT,보기6_CNT,보기7_CNT,보기8_CNT,보기9_CNT,보기10_CNT,source_file
0,여러분은 평소 체력 관리를 위해 어떤 활동을 하고 계신가요? 모두 선택해주세요.,헬스,홈트레이닝,요가/필라테스,달리기/걷기,자전거 타기,"스포츠(축구, 배드민턴 등)",수영,등산,기타,...,677.0,332.0,2035.0,472.0,299.0,255.0,598.0,212.0,1061.0,qpoll_join_250106.xlsx
1,여러분은 다가오는 여름철 가장 걱정되는 점이 무엇인가요?,더위와 땀,피부 트러블,냉방병,전기요금 부담,체력 저하,휴가 계획 스트레스,특별히 걱정되는 것이 없다,NaN,NaN,...,299.0,284.0,856.0,558.0,247.0,244.0,NaN,NaN,NaN,qpoll_join_250604.xlsx
2,여러분의 휴대폰 갤러리에 가장 많이 저장되어져 있는 사진은 무엇인가요?,셀카/인물 사진,음식 사진,풍경/여행 사진,반려동물 사진,메모용 캡처/스크린샷,친구/가족과의 단체 사진,SNS/인터넷에서 저장한 이미지,"업무/학업 관련 사진(자료, 필기 등)",기타,...,309.0,1013.0,299.0,417.0,1018.0,334.0,411.0,270.0,NaN,qpoll_join_250716.xlsx
3,여러분은 최근 가장 지출을 많이 한 곳은 어디입니까?,외식비,배달비,옷/쇼핑,"콘서트, 전시 등 문화생활",기타,NaN,NaN,NaN,NaN,...,568.0,953.0,407.0,986.0,NaN,NaN,NaN,NaN,NaN,qpoll_join_250624.xlsx
4,여러분은 올해 해외여행을 간다면 어디로 가고 싶나요? 모두 선택해주세요,유럽,동남아,미국/캐나다,일본/중국,기타,해외여행을 가고싶지 않다,NaN,NaN,NaN,...,1233.0,1023.0,1228.0,207.0,786.0,NaN,NaN,NaN,NaN,qpoll_join_250326.xlsx


## 👥 4. 패널 정보 해독표 생성

Welcome 파일에서 추출된 패널 정보를 정리하여 패널 정보 해독표를 생성합니다.

In [43]:
# 3. '패널 정보 해독표 (Panel Data Dictionary)' 추출 및 정리 (⭐️ 완전히 새로 작성된 로직)

print("=== 패널 정보 해독표 생성 시작 ===")

# 'Welcome'이 포함된 행만 필터링
panel_raw_df = master_df[master_df['source_file'].str.contains('Welcome', na=False)].copy()

# 최종 데이터를 담을 리스트
panel_data_list = []

# 현재 처리 중인 변수 정보를 저장할 변수
current_variable_info = {
    'variable': None,
    'description': None,
    'type': None
}

# 한 줄씩 순회하며 데이터 구조를 직접 파싱
for index, row in panel_raw_df.iterrows():
    # Case 1: 변수 정의 행 (예: Q1, 결혼여부, SINGLE)
    # '문항유형' 컬럼에 값이 있는 행을 변수 정의의 시작으로 판단
    if pd.notna(row['문항유형']):
        current_variable_info['variable'] = row['변수명']
        current_variable_info['description'] = row['문항']
        current_variable_info['type'] = row['문항유형']
        
        # 만약 code, label이 없는 단일 값 변수라면 (예: mb_sn, 패널ID)
        if current_variable_info['variable'] != current_variable_info['description']:
            panel_data_list.append({
                'variable': current_variable_info['variable'],
                'description': current_variable_info['description'],
                'type': current_variable_info['type'],
                'code': np.nan, # 코드가 없는 경우
                'label': np.nan # 라벨이 없는 경우
            })

    # Case 2: 코드와 라벨 정의 행 (예: 1, 미혼)
    # '문항유형'은 비어있고, '변수명'(code)과 '문항'(label)에 값이 있는 행
    elif pd.isna(row['문항유형']) and pd.notna(row['변수명']) and pd.notna(row['문항']):
        panel_data_list.append({
            'variable': current_variable_info['variable'],
            'description': current_variable_info['description'],
            'type': current_variable_info['type'],
            'code': row['변수명'],
            'label': row['문항']
        })

# 리스트를 데이터프레임으로 변환
panel_dict_df = pd.DataFrame(panel_data_list)

# Numeric 타입 변수(자녀수)처럼 코드/라벨이 없는 행 제거
panel_dict_df = panel_dict_df.dropna(subset=['label'])

# 결과 저장
panel_dict_output_path = '../data/processed/panel_data_dictionary.csv'
panel_dict_df.to_csv(panel_dict_output_path, index=False, encoding='utf-8-sig')

print(f"✅ 패널 정보 해독표 저장 완료 (수정됨): {panel_dict_output_path}")
print(f"📊 최종 데이터 크기: {panel_dict_df.shape}")
print(f"📋 패널 정보 해독표 미리보기:")
display(panel_dict_df.head())

=== 패널 정보 해독표 생성 시작 ===
✅ 패널 정보 해독표 저장 완료 (수정됨): ../data/processed/panel_data_dictionary.csv
📊 최종 데이터 크기: (578, 5)
📋 패널 정보 해독표 미리보기:


,variable,description,type,code,label
2,Q1,결혼여부,SINGLE,1,미혼
3,Q1,결혼여부,SINGLE,2,기혼
4,Q1,결혼여부,SINGLE,3,기타(사별/이혼 등)
7,Q3,가족수,SINGLE,1,1명(혼자 거주)
8,Q3,가족수,SINGLE,2,2명


## ✅ 5. 결과 확인

생성된 파일들을 확인하고 최종 결과를 검증합니다.

In [44]:
# 생성된 파일들 확인
print("=== 생성된 파일 확인 ===")

# 설문 요약본 확인
if os.path.exists('../data/processed/quickpoll_summary.csv'):
    quickpoll_check = pd.read_csv('../data/processed/quickpoll_summary.csv')
    print(f"✅ 설문 요약본: {quickpoll_check.shape} (행, 열)")
    print(f"   - 컬럼 수: {len(quickpoll_check.columns)}")
    print(f"   - 파일 크기: {os.path.getsize('../data/processed/quickpoll_summary.csv')} bytes")
else:
    print("❌ 설문 요약본 파일을 찾을 수 없습니다.")

# 패널 정보 해독표 확인
if os.path.exists('../data/processed/panel_data_dictionary.csv'):
    panel_check = pd.read_csv('../data/processed/panel_data_dictionary.csv')
    print(f"✅ 패널 정보 해독표: {panel_check.shape} (행, 열)")
    print(f"   - 컬럼 수: {len(panel_check.columns)}")
    print(f"   - 파일 크기: {os.path.getsize('../data/processed/panel_data_dictionary.csv')} bytes")
else:
    print("❌ 패널 정보 해독표 파일을 찾을 수 없습니다.")

print(f"\n🎉 해독표 정리 완료!")
print(f"📁 생성된 파일들:")
print(f"   1. quickpoll_summary.csv - 설문 요약본")
print(f"   2. panel_data_dictionary.csv - 패널 정보 해독표")
print(f"\n💡 이제 이 해독표들을 활용하여 데이터 분석을 진행할 수 있습니다!")

=== 생성된 파일 확인 ===
✅ 설문 요약본: (41, 23) (행, 열)
   - 컬럼 수: 23
   - 파일 크기: 13242 bytes
✅ 패널 정보 해독표: (578, 5) (행, 열)
   - 컬럼 수: 5
   - 파일 크기: 29916 bytes

🎉 해독표 정리 완료!
📁 생성된 파일들:
   1. quickpoll_summary.csv - 설문 요약본
   2. panel_data_dictionary.csv - 패널 정보 해독표

💡 이제 이 해독표들을 활용하여 데이터 분석을 진행할 수 있습니다!
